![lsbu](https://storage-prtl-co.imgix.net/endor/organisations/11506/logos/1540893470_rsz_lsbu_crest_simple_white_text_horizon_blackoutline.jpg)

## Introduction
This notebook shows how to configure the flask application, implementing the `Attendance monitoring via SMS` project. 

## Objectives
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

## Configuartion

Make sure you have the required environments and packages configured, as mentioned in the `01a_set_up_the_environments
` notebook.

## Import Libraries

In [ ]:
import os
import re
import pathlib
import secrets
import pandas as pd
from typing import Dict, Optional
from flask import Flask, request, render_template, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from bs4 import BeautifulSoup
import altair as alt
from flask_mail import Mail, Message
from dotenv import load_dotenv
from datetime import date, datetime
from twilio.rest import Client

## Folder Structure
```python
|-- attendance_monioring_via_sms
|   |-- .twilio
|   |-- datasets
|      |-- *.csv
|   |-- docs
|      |-- *.md
|      |-- *.pdf
|   |-- env_vars
|      |-- .env.txt
|   |-- images
|      |-- *.png
|   |-- nbs
|      |-- *.ipynb 
|   |-- static
|      |- *.css
|      |- *.js
|   |-- templates
|      |- *.html
|   |-- app.py
|   |-- conda_env_twilio.yml
|   |-- requirements.txt

```

## Set Paths

In [ ]:
# Set paths
base_dir = pathlib.Path().absolute().parent

# Path to all data directories
data = pathlib.Path(base_dir/'datasets')

# Path to credentials data
credentials = data/'credentials'

# Path to raw data
raw = data/'raw'

# Path to processed data
processed = data/'processed'

# Path to environment variables
env_vars = pathlib.Path(base_dir/'env_vars')

## Load Environment Vaiables
All the sensitive user information are stored in a `.env.txt` file, which is loaded, using `dotenv`.

In [ ]:
# Load dotenv file
load_dotenv(env_vars/'.env.txt')

## Configure Flask App
The flask app has a flask-mail app to send emails, which is configured with an `smtp` based mail testing software/website called, `Mailtrap`. It creats dummy emails, sender and receiver rather than acutally sending the emails but with the same conventions that apply to the real emails.

The flask_login app tracks the login of the users to the application and used for session management and provide the endpoints from unauthorised access by an adversary.

In [ ]:
# Flask app
app = Flask(__name__)
# Configure a secret-key for the flask app
app.config["SECRET_KEY"] = secrets.token_hex(24)

# Configure flask mail app
app.config['MAIL_SERVER'] = 'smtp.mailtrap.io'
app.config['MAIL_PORT'] = 2525
app.config['MAIL_USERNAME'] = os.environ.get('MAIL_USERNAME')
app.config['MAIL_PASSWORD'] = os.environ.get('MAIL_PASSWORD')
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False

# Create login_manager app
login_manager = LoginManager(app)
# Create mail app
mail = Mail(app)

# Create twilio client object
account_sid = os.environ.get("TWILIO_ACCOUNT_SID")
auth_token = os.environ.get("TWILIO_AUTH_TOKEN")
client = Client(account_sid, auth_token)

# Define a User class to store all the users from the .csv input files
users: Dict[str, "User"] = {}

class User(UserMixin):
    def __init__(self, id: str, username: str, email: str, password: str):
        self.id = id
        self.username = username
        self.email = email
        self.password = password

    @staticmethod
    def get(user_id: str) -> Optional["User"]:
        return users.get(user_id)

    def __str__(self) -> str:
        return f"<Id: {self.id}, Username: {self.username}, Email: {self.email}>"

    def __repr__(self) -> str:
        return self.__str__()

data = pd.read_csv(credentials/'login_credentials.csv')
for index in data.index:
    users[str(index)] = User(
        id=index,
        username=data.loc[index, "first_name"],
        email=data.loc[index, "email"],
        password=data.loc[index, "password"],
    )

###
"""
All the endpoints for the application goes here
"""
###

if __name__ == '__main__':
    app.run(debug=True)

## Endpoint for Login and Logout

In [ ]:
# Login endpoint
@login_manager.user_loader
def load_user(user_id: str) -> Optional[User]:
    return User.get(user_id)

@login_manager.unauthorized_handler
def unauthorized():
    return redirect(url_for('login'))

@app.route('/', methods=['GET', 'POST'])
def login():
    error = None
    # Load data
    df_logins = pd.read_csv(credentials/'login_credentials.csv')
    if request.method == 'POST':
        if request.is_json:
            email = str(request.json['email'])
            password = str(request.json['password'])
        else:
            email = str(request.form['email'])
            password = str(request.form['password'])
            if email=='':
                error='Email Required!'
                return render_template('login.html', error=error)
            elif password=='':
                error='Password Required!'
                return render_template('login.html', error=error)
        if df_logins.loc[df_logins.email==email, 'password'].empty:
            test = False
        else:
            test = (df_logins.email.isin([email]).any()) and (df_logins.loc[df_logins.email==email, 'password'].astype(str).values[0]==password)

        if test: 
            user_id = df_logins.loc[df_logins.email==email].index[0]
            user = User.get(str(user_id))
            login_user(user)
            
            return redirect(url_for('home'))
        else:
            error = 'Invalid Credentials. Please try again. If you do not have an account, please create an account first then try logging in.'
    return render_template('login.html', error=error)

# Endpoint for logout
@app.route("/logout")
@login_required
def logout():
    logout_user()
    return redirect(url_for("login"))

## Endpoint for Registration to he App

In [ ]:
# Register endpoint
@app.route('/register', methods=['GET', 'POST'])
def register():
    message = None
    # Load Data
    df_logins = pd.read_csv(credentials/'login_credentials.csv')
    df_admins = pd.read_csv(credentials/'admins.csv')
    if request.method == 'POST':
        email = str(request.form['email'])
        test = df_logins.email.isin([email]).any()
        if test:
            message = 'This email already exists. Please click \'Forgotten Password\'.'
            return render_template('register.html', message=message)
        elif request.form['first_name'] == '':
            message = 'First Name is required!'
            return render_template('register.html', message=message)
        elif request.form['last_name'] == '':
            message = 'Last Name is required!'
            return render_template('register.html', message=message)
        elif request.form['email'] == '':
            message = 'Email is required!'
            return render_template('register.html', message=message)
        elif request.form['password'] == '':
            message = 'Password is required!'
            return render_template('register.html', message=message)
        elif request.form['password'] != request.form['confirm_password']:
            message = 'Password should match!'
            return render_template('register.html', message=message)
        elif df_admins.email.isin([str(request.form['email'])]).any()==False:
            message = 'Email not allowed! Please make sure the email entered is in the admin list or contact the administrator!'
            return render_template('register.html', message=message)
        else:
            first_name = request.form['first_name']
            last_name = request.form['last_name']
            password = request.form['password']
            
            df = df_logins.append({'first_name': first_name, 'last_name': last_name, 'email': email, 'password': password}, ignore_index=True)
            df.to_csv(credentials/'login_credentials.csv', index=False)
            message = "User created successfully. Please go back to log in."
            return render_template('register.html', message=message)
    return render_template('register.html')

## Endpoint for Retrieving Password

In [ ]:
# Endpoint for retrivieng password
@app.route('/retpass', methods=['GET', 'POST'])
def retpass():
    message = None
    # Load data
    df_logins = pd.read_csv(credentials/'login_credentials.csv')
    if request.method == 'POST':
        email = request.form['email']
        user = df_logins.email.isin([str(email)]).any()
        if user:
            msg = Message("Your Password Retrieval Requst..",
                          sender="admin@attendance.lsbu.ac.uk", recipients=[email])
            msg.body = "Hi,\n\nThanks for your password retirval request!\n\nYour password is: " +\
                df_logins.loc[df_logins.email==email, 'password'].astype(str).values[0] + "\n\nThanks for using our service.\n\nRegards,\nAdmin Team"
            mail.send(msg)
            message = "Password sent to " + email
            return render_template('retpass.html', message=message)
        elif request.form['email'] == '':
            message = 'Email is required!'
            return render_template('retpass.html', message=message)
        else:
            message = "This email doesn't exist. Please create an account."
            return render_template('retpass.html', message=message)
    return render_template('retpass.html')

## Endpoint for Home Page

In [ ]:
# Endpoint for home
@app.route('/home')
@login_required
def home():
    user_name = current_user.username
    df = pd.read_csv(processed/'weekly_attendance.csv')
    df.course_code = df.course_code.astype(str)
    df.academic_semester = df.academic_semester.astype(str)

    rect = alt.Chart(df).mark_rect().encode(
        alt.X('course_code', bin=False, title='Course Code'),
        alt.Y('total_attendees', bin=False, title='Total Attendance (Weekly)'),
        alt.Color('total_attendees',
                scale=alt.Scale(scheme='greenblue'),
                legend=alt.Legend(title='Total Attendees')
                )
    ).properties(height=500, width=578, title='Total Attendees (Weekly) by Course')

    circ = rect.mark_point().encode(
        alt.ColorValue('grey'),
        alt.Size('total_attendees',
                legend=alt.Legend(title='Total Attendees')
                ))

    chart1 = alt.vconcat(
        rect + circ
    ).resolve_legend(
        color="independent",
        size="independent"
    )

    chart2 = alt.Chart(df).mark_circle(size=200).encode(
        x=alt.X('course_code', title='Course Code'),
        y=alt.Y('total_attendees', title='Total Attendance (Weekly)'),
        color=alt.Color('academic_semester', title='Semester'),
        tooltip=['course_code', 'course_name', 'month', 'year', 'academic_semester', 'academic_week', 'total_attendees']
    ).interactive().properties(height=500,
                            width=578, title='Total Attendees (Weekly) by Course')

    (chart1 | chart2).save('templates/charts.html')

    altair_bs = BeautifulSoup(
        open('templates/charts.html').read(), 'html.parser')

    altair_script = altair_bs.find_all('script')[-1].contents[0]

    chart_id = altair_bs.find('body').div['id']

    with open('static/plots.js', 'w')as f:
        f.write(altair_script)
        f.close()

    return render_template('home.html', chart_id=chart_id, user_name=user_name)

## Endpoint for About and Contact Page

In [ ]:
# Endpoint for about
@app.route('/about')
@login_required
def about():
    return render_template('about.html')

# Endpoint for contact
@app.route('/contact')
@login_required
def contact():
    return render_template('contact.html')

## Endpoint for Showing Data in Tables

In [ ]:
# Endpoint for Student Table
@app.route('/table1', methods=['GET'])
@login_required
def table1():  
    df = pd.read_csv(credentials/'students.csv').dropna()
    
    headings = df.columns.values
    headings = tuple(headings)

    data = df.to_records(index=False)
    data = tuple(data)
    return render_template('table1.html', headings=headings, data=data)


# Endpoint for Admin Table
@app.route('/table2', methods=['GET'])
@login_required
def table2():  # put application's code here

    df = pd.read_csv(credentials/'admins.csv').dropna()

    headings = df.columns.values
    headings = tuple(headings)

    data = df.to_records(index=False)
    data = tuple(data)
    return render_template('table2.html', headings=headings, data=data)

# Endpoint for Course Table
@app.route('/table3', methods=['GET'])
@login_required
def table3():  # put application's code here

    df = pd.read_csv(credentials/'courses.csv').dropna()

    headings = df.columns.values
    headings = tuple(headings)

    data = df.to_records(index=False)
    data = tuple(data)
    return render_template('table3.html', headings=headings, data=data)

# Endpoint for Timetable Table
@app.route('/table4', methods=['GET'])
@login_required
def table4():  # put application's code here

    df = pd.read_csv(credentials/'timetable.csv').dropna()

    headings = df.columns.values
    headings = tuple(headings)

    data = df.to_records(index=False)
    data = tuple(data)
    return render_template('table4.html', headings=headings, data=data)

# Endpoint for Attendance Statistics Table
@app.route('/table5', methods=['GET'])
@login_required
def table5():  # put application's code here

    df = pd.read_csv(processed/'weekly_attendance.csv').dropna()

    headings = df.columns.values
    headings = tuple(headings)

    data = df.to_records(index=False)
    data = tuple(data)
    return render_template('table5.html', headings=headings, data=data)

The other endpoints are discussed in the later notebooks, which involves some data engineering steps to process, extract, and generate datasets, using various techniques.

### Summary
In this notebook it was demonstrated how to:
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

<center><b>Author</b></center>

| Name | Date Created | Last Modified |
|------|--------------|---------------|
|Khaled Ahmed | 30/01/2023 | 30/01/2023|